In [154]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [67]:
just = pd.read_csv('justice_results.csv')
just.head()

,docketId,caseId,docket,justice,petitioner_vote,petitioner_pitch,respondent_pitch,lcDispositionDirection,justiceName,lagged_ideology,...,respondent_harvard_pos,petitioner_harvard_neg,respondent_harvard_neg,petitioner_liwc_pos,respondent_liwc_pos,petitioner_liwc_neg,respondent_liwc_neg,term,pitch_diff,conservative_lc
0,1982-038-01,1982-038,81-1802,104,1,0.098386,0.547079,2,SDOConnor,1.491,...,1.0,0.0,1.0,3.0,0.0,1.0,0.0,1982,-0.448692,0.0
1,1982-038-01,1982-038,81-1802,102,1,-0.548233,-0.392148,2,WHRehnquist,4.036,...,7.0,3.0,3.0,2.0,2.0,0.0,1.0,1982,-0.156086,0.0
2,1982-038-01,1982-038,81-1802,99,1,-1.329200,-0.881118,2,WEBurger,1.491,...,18.0,1.0,8.0,1.0,11.0,0.0,2.0,1982,-0.448082,0.0
3,1982-038-01,1982-038,81-1802,103,1,-0.327295,0.031076,2,JPStevens,-0.265,...,7.0,1.0,2.0,2.0,2.0,2.0,1.0,1982,-0.358370,0.0
4,1982-038-01,1982-038,81-1802,98,1,0.008952,-0.011692,2,TMarshall,-3.684,...,4.0,1.0,2.0,0.0,3.0,1.0,0.0,1982,0.020644,0.0


In [68]:
# DAL
just['dal_petpos'] = just['petitioner_dal_pos']/just['petitioner_wc'] - just['respondent_dal_pos']/just['respondent_wc']
just['dal_petneg'] = just['petitioner_dal_neg']/just['petitioner_wc'] - just['respondent_dal_neg']/just['respondent_wc']
# Harvard
just['harvard_petpos'] = just['petitioner_harvard_pos']/just['petitioner_wc'] - just['respondent_harvard_pos']/just['respondent_wc']
just['harvard_petneg'] = just['petitioner_harvard_neg']/just['petitioner_wc'] - just['respondent_harvard_neg']/just['respondent_wc']
# LIWC
just['liwc_petpos'] = just['petitioner_liwc_pos']/just['petitioner_wc'] - just['respondent_liwc_pos']/just['respondent_wc']
just['liwc_petneg'] = just['petitioner_liwc_neg']/just['petitioner_wc'] - just['respondent_liwc_neg']/just['respondent_wc']
# Questions
just['pet_questions'] = just['petitioner_count'] - just['respondent_count']
# ideology x lower court conservative
just['idealxLC'] = just['lagged_ideology'] * just['conservative_lc']

In [69]:
# subset to necessary columns
just = just[['petitioner_vote',
      'pitch_diff', 
      'dal_petpos',
      'dal_petneg',
      'harvard_petpos',
      'harvard_petneg',
      'liwc_petpos',
      'liwc_petneg',
      'pet_questions',
      'conservative_lc',
      'lagged_ideology',
      'idealxLC',
      'sgpetac',
      'sgrespac',
      'petac',
      'respac',
      'petNumStat',
      'respNumStat',
      'justiceName'      
     ]]

In [70]:
# dummy variables for justices
just = pd.get_dummies(just, columns = ['justiceName'])

In [71]:
just.to_csv('justice_results_clean.csv', index = False)

## Original Model

Notice there is missing data that gets dropped between different iterations of the models. This inconsistency could result in a distributional shift.

In [72]:
just = just.dropna()

In [139]:
y = just['petitioner_vote']
X = just.iloc[:,1:]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state = 1)

mod1 = LogisticRegression(random_state = 1, solver = 'liblinear').fit(X_train, y_train)

# majority class classifier
print(y_test.mean())
print(mod1.score(X_test, y_test))

0.5334672021419009
0.6204819277108434


## Refining measures

#### DAL

In [131]:
dal = just.drop(labels = ['harvard_petpos', 'harvard_petneg', 'liwc_petpos', 'liwc_petneg'], axis = 1)

y = dal['petitioner_vote']
X = dal.iloc[:,1:]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state = 1)

mod2= LogisticRegression(random_state = 1, solver = 'liblinear').fit(X_train, y_train)
print(y_test.mean())
print(mod2.score(X_test, y_test))

0.5334672021419009
0.6204819277108434


#### Harvard

In [134]:
harvard = just.drop(labels = ['dal_petpos', 'dal_petneg', 'liwc_petpos', 'liwc_petneg'], axis = 1)

y = harvard['petitioner_vote']
X = harvard.iloc[:,1:]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state = 1)

mod3= LogisticRegression(random_state = 1, solver = 'liblinear').fit(X_train, y_train)
print(y_test.mean())
print(mod3.score(X_test, y_test))

0.5334672021419009
0.6211512717536813


#### LIWC

In [135]:
liwc = just.drop(labels = ['dal_petpos', 'dal_petneg', 'harvard_petpos', 'harvard_petneg'], axis = 1)

y = liwc['petitioner_vote']
X = liwc.iloc[:,1:]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state = 1)

mod4= LogisticRegression(random_state = 1, solver = 'liblinear').fit(X_train, y_train)
print(y_test.mean())
print(mod4.score(X_test, y_test))

0.5334672021419009
0.6231593038821954


In [108]:
# get model coefficients
dict(zip(mod4.feature_names_in_, mod4.coef_[0]))

{'pitch_diff': -0.21277442198119592,
 'liwc_petpos': -0.5839041403081796,
 'liwc_petneg': -0.45279296676773606,
 'pet_questions': -0.058790223315654,
 'conservative_lc': -0.01835391760323113,
 'lagged_ideology': -0.08160147456831034,
 'idealxLC': -0.23582846968665158,
 'sgpetac': 0.4745546471680834,
 'sgrespac': -0.6383512554322763,
 'petac': 0.04556998001917423,
 'respac': -0.06742169937467193,
 'petNumStat': 0.03502738580940508,
 'respNumStat': -0.0153239575993714,
 'justiceName_AMKennedy': 0.4341542045871641,
 'justiceName_AScalia': 0.5761031674948129,
 'justiceName_BRWhite': 0.1131319814640918,
 'justiceName_CThomas': -0.5046518860767433,
 'justiceName_DHSouter': -0.12110367724243022,
 'justiceName_EKagan': -0.19088138920310935,
 'justiceName_HABlackmun': -0.5746266028243279,
 'justiceName_JGRoberts': 0.529903313225748,
 'justiceName_JPStevens': -0.7363636468387532,
 'justiceName_LFPowell': 0.4835493519992183,
 'justiceName_RBGinsburg': -0.4086669549493009,
 'justiceName_SAAlito': 

## Feature Importance W/ Logistic Regression

When you run a regression, what is the null model? i.e. what is the null hypothesis?
What are the stars telling you? What is the coefficient "significantly" different from?

The null model is that nothing has any effect on the DGP. That's spectacularly unlikely.

What's the null model they assume in the supreme court paper? Fixed effects and fixed effects + pitch.

Is this a good null model if you're trying to show that vocal pitch is predictive? Why or why not?

What might be a more appropriate null model?

In [158]:
# Fixed effects only
y = liwc['petitioner_vote']
X = liwc.iloc[:,14:]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state = 1)

mod4= LogisticRegression(random_state = 1, solver = 'liblinear').fit(X_train, y_train)
print(y_test.mean())
print(mod4.score(X_test, y_test))

0.5334672021419009
0.536813922356091


In [159]:
# Fixed effects and pitch
y = liwc['petitioner_vote']
X = liwc.iloc[:,np.r_[1,14:32]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state = 1)

mod4= LogisticRegression(random_state = 1, solver = 'liblinear').fit(X_train, y_train)
print(y_test.mean())
print(mod4.score(X_test, y_test))

0.5334672021419009
0.5522088353413654


In [160]:
# LIWC Model without pitch difference
y = liwc['petitioner_vote']
X = liwc.iloc[:,2:]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state = 1)

mod4= LogisticRegression(random_state = 1, solver = 'liblinear').fit(X_train, y_train)
print(y_test.mean())
print(mod4.score(X_test, y_test))

0.5334672021419009
0.6251673360107095


In [162]:
# LIWC Model without pitch and positive/negative words
y = liwc['petitioner_vote']
X = liwc.iloc[:,4:]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state = 1)

mod4= LogisticRegression(random_state = 1, solver = 'liblinear').fit(X_train, y_train)
print(y_test.mean())
print(mod4.score(X_test, y_test))

0.5334672021419009
0.6238286479250335


## Feature importance with Random Forest

What's the difference between effect size and feature importance? Is a regression coefficient feature importance?

In [167]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance

In [172]:
# Full model
y = liwc['petitioner_vote']
X = liwc.iloc[:,1:]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state = 1)

liwcforest = RandomForestClassifier(random_state=0).fit(X_train, y_train)

liwcforest.score(X_test, y_test)

0.6653279785809906

In [186]:
# Without pitch
y = liwc['petitioner_vote']
X = liwc.iloc[:,2:]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state = 1)

liwcforest = RandomForestClassifier(random_state=0).fit(X_train, y_train)

liwcforest.score(X_test, y_test)

0.6693440428380187

In [187]:
# Without emotion
y = liwc['petitioner_vote']
X = liwc.iloc[:,4:]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state = 1)

liwcforest = RandomForestClassifier(random_state=0).fit(X_train, y_train)

liwcforest.score(X_test, y_test)

0.6847389558232931

In [188]:
# Fixed effect only
y = liwc['petitioner_vote']
X = liwc.iloc[:,12:]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state = 1)

liwcforest = RandomForestClassifier(random_state=0).fit(X_train, y_train)

liwcforest.score(X_test, y_test)

0.5488621151271754

In [185]:
%%time
liwcimport = permutation_importance(
    liwcforest, X_test, y_test, n_repeats=10, random_state=1, n_jobs=2
)

dict(zip(liwcforest.feature_names_in_, result.importances_mean))

CPU times: total: 1.56 s
Wall time: 4.36 s


{'pitch_diff': 0.016465863453815198,
 'liwc_petpos': 0.004551539491298484,
 'liwc_petneg': 0.004484605087014681,
 'pet_questions': 0.02463186077643902,
 'conservative_lc': -0.0006024096385542687,
 'lagged_ideology': 0.005622489959839305,
 'idealxLC': 0.007496653279785759,
 'sgpetac': 0.02489959839357423,
 'sgrespac': 0.025100401606425647,
 'petac': 0.031057563587684012,
 'respac': 0.04089692101740291,
 'petNumStat': 0.019611780455153882,
 'respNumStat': 0.029585006693440375,
 'justiceName_AMKennedy': -0.001874163319946487,
 'justiceName_AScalia': -0.00046854082998665225,
 'justiceName_BRWhite': -6.69344042838027e-05,
 'justiceName_CThomas': 0.0,
 'justiceName_DHSouter': -0.001004016064257074,
 'justiceName_EKagan': -0.0006024096385542355,
 'justiceName_HABlackmun': 0.0,
 'justiceName_JGRoberts': -0.004551539491298584,
 'justiceName_JPStevens': 0.0013386880856759876,
 'justiceName_LFPowell': 0.0,
 'justiceName_RBGinsburg': -0.0028112449799197136,
 'justiceName_SAAlito': 0.00087014725568

## Over and under fitting

What causes overfitting?
- lack of training data not representing the entire distribution of data
- over-specified model is exploiting artifacts in the training data

Solutions:
- Increase the training data
- cross validation (kind of)
- use a more parsimonious model
- regularization

In [211]:
from sklearn.linear_model import LogisticRegressionCV

In [201]:
# Full model
y = liwc['petitioner_vote']
X = liwc.iloc[:,1:]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .1, random_state = 1) # manipulate the size of the training data set

liwcforest = RandomForestClassifier(random_state=0).fit(X_train, y_train)

liwcforest.score(X_test, y_test)

0.6827309236947792

In [226]:
liwc = just.drop(labels = ['dal_petpos', 'dal_petneg', 'harvard_petpos', 'harvard_petneg'], axis = 1)

y = liwc['petitioner_vote']
X = liwc.iloc[:,1:]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state = 1)

mod4= LogisticRegressionCV(cv = 5, random_state = 1, solver = 'liblinear').fit(X_train, y_train)
print(y_test.mean())
print(mod4.score(X_test, y_test))

0.5334672021419009
0.6251673360107095
